In [1]:
import pandas as pd
import ta
import matplotlib.pyplot as plt
import numpy as np

<div class="alert alert-block alert-info">
<b>Data Preprocessing</b>

1. Load the data: 'Date', 'Dernier', 'Ouv.', ' Plus Haut', 'Plus Bas', 'Vol.', 'Variation %'
2. From 'Dernier' make new columns: 'rsi', 'macd', 'ema20', 'ema50'
3. Convert the data with K, M, % to numeric: 'Ouv.', ' Plus Haut', 'Plus Bas', 'Vol.', 'Variation %'
4. Fill the 'Vol.' column with the median of that column
5. Drop the first 50 rows which have NaNs


Output: df.shape = (1568, 11)


</div>

In [2]:
#------------------------------------------------#
# All the functions are in the Features.py file
# This is the full pipeline to preprocess the data
# The cells below are the steps of the pipeline
#------------------------------------------------#

from src.Features import FeatureEngineer

gaz = pd.read_csv('Data/Dutch TTF Natural Gas Futures - Données Historiques (1).csv')
print(gaz.shape) # (1617, 11)

fe_gaz = FeatureEngineer(gaz)
fe_gaz.apply_preprocessing()

print(fe_gaz.df.shape)
fe_gaz.df.head() # (1568, 11)

#------------------------------------------------#

(1617, 7)


/Users/pierre/Documents/GitHub/Reinforcement-Learning-Project/src/Features.py:34: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  self.df['Date']=pd.to_datetime(self.df['Date'])


(1568, 11)


,Date,Dernier,Ouv.,Plus Haut,Plus Bas,Vol.,Variation %,rsi,macd,ema20,ema50
1567,1514937600000000000,19.325,19.325,19.325,19.325,210.0,0.0003,0.340145,0.427470,0.062645,0.070533
1566,1515024000000000000,19.200,19.200,19.200,19.200,210.0,-0.0065,0.322674,0.427887,0.062343,0.070431
1565,1515110400000000000,18.915,18.915,18.915,18.915,210.0,-0.0148,0.284096,0.427832,0.061958,0.070277
1564,1515369600000000000,19.050,19.050,19.050,19.050,210.0,0.0071,0.313652,0.428520,0.061663,0.070156
1563,1515456000000000000,19.435,19.435,19.435,19.435,210.0,0.0202,0.392683,0.430260,0.061547,0.070114


---

In [ ]:
gaz = pd.read_csv('Data/Dutch TTF Natural Gas Futures - Données Historiques (1).csv')
gaz['Date']=pd.to_datetime(gaz['Date'])
lists = ['Dernier', 'Ouv.', ' Plus Haut', 'Plus Bas', 'Vol.','Variation %']
for i in lists: 
    gaz[i] = gaz[i].str.replace(',', '.')
print(gaz.columns)

# print the data types of the columns
print(gaz.dtypes)

# order the data by date
gaz = gaz.sort_values(by='Date')
gaz = gaz.reset_index(drop=True)

gaz.head()

In [ ]:
gaz['Dernier'] = pd.to_numeric(gaz['Dernier'], errors='coerce')

gaz['rsi'] = ta.momentum.rsi(gaz['Dernier'], window=14) # Relative Strength Index
gaz['macd'] = ta.trend.macd_diff(gaz['Dernier']) # Moving Average Convergence Divergence
gaz['ema20'] = ta.trend.ema_indicator(gaz['Dernier'], window=20) # Exponential Moving Average
gaz['ema50'] = ta.trend.ema_indicator(gaz['Dernier'], window=50) # Exponential Moving Average

print(gaz.columns)
print(gaz.shape)
gaz.head()

In [ ]:
def convert_k_m_to_numeric(value):
    """
    Convert values with 'K' or 'M' suffix to float numbers.
    Args:
    - value: The string or numeric value to convert.
    
    Returns:
    - The converted value as float if 'K' or 'M' was found; otherwise, the original value.
    """
    if isinstance(value, str):  # Only process strings
        if value.endswith('K'):
            return float(value[:-1]) * 1e3
        elif value.endswith('M'):
            return float(value[:-1]) * 1e6
        elif value.endswith('%'):
            return float(value[:-1]) / 100
    return float(value)

In [ ]:
missing_values = gaz.isnull().mean() * 100
print(missing_values)

In [ ]:
# convert columns 'Ouv.', 'Plus Haut', 'Plus Bas', 'Vol.', 'Variation %' to numeric

gaz['Ouv.'] = gaz['Ouv.'].apply(convert_k_m_to_numeric)
gaz[' Plus Haut'] = gaz[' Plus Haut'].apply(convert_k_m_to_numeric)
gaz['Plus Bas'] = gaz['Plus Bas'].apply(convert_k_m_to_numeric)
gaz['Vol.'] = gaz['Vol.'].apply(convert_k_m_to_numeric)
gaz['Variation %'] = gaz['Variation %'].apply(convert_k_m_to_numeric)

missing_values = gaz.isnull().mean() * 100
print(missing_values)

gaz.head()




# gaz['Vol.'] = gaz['Vol.'].apply(convert_k_m_to_numeric)
# for column in gaz.columns:
#     gaz[column] = pd.to_numeric(gaz[column], errors='coerce')
# assert gaz.applymap(np.isreal).all().all(), "Non-numeric data found in the dataset."

In [ ]:
# Fill 'Vol.' with median
gaz['Vol.'] = gaz['Vol.'].fillna(gaz['Vol.'].median())

print(gaz.shape)
# drop rows with missing values
gaz = gaz.dropna()

missing_values = gaz.isnull().mean() * 100
print(missing_values)
print(gaz.shape)


# convert 'Date' to numeric
gaz['Date'] = pd.to_numeric(gaz['Date'])

gaz.head()

In [ ]:
from src.Features import FeatureEngineer

feature_engineer = FeatureEngineer(gaz)

feature_engineer.calculate_technical_indicators()  # Calculate RSI, MACD, EMA20, EMA50

feature_engineer.normalize_features()  # Normalize the features
features_vector = feature_engineer.construct_feature_vector()  # Get the feature vector

In [ ]:
# make a copy of gaz and fill NaN values with 0
gaz_copy = gaz.copy()
gaz_copy.fillna(0, inplace=True)

gaz_copy.head()

In [ ]:
import numpy as np
import pandas as pd

from src.Environment import TradingEnvironment


# Example usage:

# Define a simple policy function (replace this with your PPO agent's policy for real testing)
def sample_policy(state):
    # This is a dummy policy that randomly decides to buy, hold, or sell
    return np.random.uniform(-1, 1)

# Assuming 'gaz_data' is your preprocessed DataFrame including 'Dernier' prices and any features
env = TradingEnvironment(gaz)
backtest_history = env.run_backtest(sample_policy)


In [ ]:
from src.Agent import PPOAgent
from src.ActorCritic import Actor, Critic



# Define the environment and PPO agent parameters
state_size = 100  # Assume 100 features for the state
action_size = 1  # Buy/sell quantity as a single action
action_bound = 1  # Action limit (e.g., normalized quantity between -1 and 1)

# Instantiate the PPOAgent
agent = PPOAgent(state_size=env.state_space, action_size=env.action_space, action_bound=1)

# Now you can access actor and critic through the agent object
actor_model = Actor(state_size=100, action_size=1, action_bound=1)
critic_model = Critic(state_size=100)

# summary(actor_model, input_size=(1, 100))
# summary(critic_model, input_size=(1, 101))

# critic_model.summary()
print(actor_model)
print(critic_model)

In [ ]:
import wandb
from src.Training import train_agent , run_simulation

wandb.init(project="RL Trading", config={
    "episodes": 1000,
    "learning_rate_actor": 1e-4,
    "learning_rate_critic": 1e-3,
    # Add other relevant configuration parameters here
})
config = wandb.config


env = TradingEnvironment(gaz_copy)
agent = PPOAgent(state_size=env.state_space, action_size=env.action_space, action_bound=1)

# Proceed with training and running the simulation
train_agent(env, agent)
run_simulation(env, agent)